In [122]:
# Initial imports
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [4]:
# Import loan_status.csv to Google Colab
from google.colab import files
uploaded = files.upload()

Saving applicants_data.csv to applicants_data.csv


In [123]:
# Read the CSV file into a Pandas DataFrame
df = pd.read_csv("applicants_data.csv")

# Review the DataFrame
df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [124]:
df.drop(columns=['EIN', 'NAME'], inplace=True)
df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [125]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

In [126]:
list(df.dtypes[df.dtypes == "object"].index)

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [128]:
# Create a list of the columns with categorical variables
categorical_variables = ['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']
# Use the fit_transform method from the OneHotEncoder to encode the data
encoded_data = enc.fit_transform(df[categorical_variables])

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [129]:
# Create a DataFrame with the encoded variables
encoded_df = pd.DataFrame(
    encoded_data,
    columns = enc.get_feature_names_out(categorical_variables)
)

# Display sample data
encoded_df.head()

,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [130]:
# Create a DataFrame with the columnns containing numerical variables from the original dataset
numerical_variables_df = df.drop(columns = categorical_variables)

# Review the DataFrame
numerical_variables_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL
0,1,5000,1
1,1,108590,1
2,1,5000,0
3,1,6692,1
4,1,142590,1


In [131]:
joined_df = pd.concat([numerical_variables_df, encoded_df], axis="columns", join="inner")
joined_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34295,1,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34296,1,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34297,1,5000,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [132]:
# Define the target set y using the IS_SUCCESSFUL column
y = joined_df["IS_SUCCESSFUL"]

# Display a sample of y
y[:5]

0    1
1    1
2    0
3    1
4    1
Name: IS_SUCCESSFUL, dtype: int64

In [47]:
# Define features set X by selecting all columns but Attrition_Yes and Attrition_No
X = joined_df.drop(columns=["IS_SUCCESSFUL"])

# Review the features DataFrame
X.head()

,STATUS,ASK_AMT,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [133]:
# Split the data into training and testing datasets
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [134]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [145]:
# Define the the number of inputs (features) to the model
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

number_input_features = len(X_train.iloc[0])
# Define the number of hidden nodes for the first hidden layer
# Use the mean of the number of input features plus the number of output nurons
# Use the Python floor division (//) to return the quotent
hidden_nodes_layer1 =  (number_input_features + 4) // 2

# Define the number of hidden nodes for the second hidden layer
# Use the mean of the number of hidden nodes in the first hidden layer plus the number of output nurons
# Use the Python floor division (//) to return the quotent
hidden_nodes_layer2 = (hidden_nodes_layer1 + 1) // 2

# Create the Sequential model instance
nn = Sequential()

# Add the first hidden layer specifying the number of inputs, the number of hidden nodes, and the activation function
nn.add(Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Add the second hidden layer specifying the number of hidden nodes and the activation function
nn.add(Dense(units=hidden_nodes_layer2, activation="relu"))

# Add the output layer to the model specifying the number of output neurons and activation function
nn.add(Dense(units=1, activation="sigmoid"))

In [146]:
# Display the Sequential model summary
nn.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            (None, 59)                6844      
                                                                 
 dense_31 (Dense)            (None, 30)                1800      
                                                                 
 dense_32 (Dense)            (None, 1)                 31        
                                                                 
Total params: 8,675
Trainable params: 8,675
Non-trainable params: 0
_________________________________________________________________


In [147]:
# Compile the Sequential model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [148]:
# Fit the model using 100 epochs and the training data
fit_model = nn.fit(X_train_scaled, y_train, epochs=20)

Epoch 1/20
804/804 [==============================] - 3s 3ms/step - loss: 0.0985 - accuracy: 0.9694
Epoch 2/20
804/804 [==============================] - 2s 2ms/step - loss: 0.0071 - accuracy: 0.9982
Epoch 3/20
804/804 [==============================] - 2s 2ms/step - loss: 0.0063 - accuracy: 0.9988
Epoch 4/20
804/804 [==============================] - 2s 2ms/step - loss: 0.0033 - accuracy: 0.9994
Epoch 5/20
804/804 [==============================] - 2s 2ms/step - loss: 0.0018 - accuracy: 0.9995
Epoch 6/20
804/804 [==============================] - 2s 2ms/step - loss: 0.0028 - accuracy: 0.9994
Epoch 7/20
804/804 [==============================] - 2s 2ms/step - loss: 0.0022 - accuracy: 0.9995
Epoch 8/20
804/804 [==============================] - 2s 3ms/step - loss: 0.0037 - accuracy: 0.9992
Epoch 9/20
804/804 [==============================] - 2s 2ms/step - loss: 0.0016 - accuracy: 0.9995
Epoch 10/20
804/804 [==============================] - 2s 2ms/step - loss: 0.0012 - accuracy: 0.9997

In [149]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate (X_test_scaled,y_test,verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.0045 - accuracy: 0.9997 - 449ms/epoch - 2ms/step
Loss: 0.004534862469881773, Accuracy: 0.9996501207351685


In [150]:
# Set the model's file path
file_path = "Alphabetsoup.h5"

# Export your model to an HDF5 file
nn.save(file_path)

In [151]:
# Set the model's file path
file_path = "Alphabetsoup.h5"

# Load the model to a new object
nn_imported = tf.keras.models.load_model(file_path)

In [59]:
# Import the required libraries
import tensorflow as tf
# Set the model's file path
file_path = "Alphabetsoup.h5"

# Load the model to a new object
nn_imported = tf.keras.models.load_model(file_path)

In [152]:
number_input_features = len(X_train.iloc[0])
# Define the number of hidden nodes for the first hidden layer
# Use the mean of the number of input features plus the number of output nurons
# Use the Python floor division (//) to return the quotent
hidden_nodes_layer1 = (number_input_features + 2)//3

# Define the number of hidden nodes for the second hidden layer
# Use the mean of the number of hidden nodes in the first hidden layer plus the number of output nurons
# Use the Python floor division (//) to return the quotent
hidden_nodes_layer2 = (hidden_nodes_layer1 + 2)//3

# Create the Sequential model instance
nn_imported = Sequential()

# Add the first hidden layer specifying the number of inputs, the number of hidden nodes, and the activation function
nn_imported.add(Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Add the second hidden layer specifying the number of hidden nodes and the activation function
nn_imported.add(Dense(units=hidden_nodes_layer2, activation="relu"))

# Add the output layer to the model specifying the number of output neurons and activation function
nn_imported.add(Dense(units=1, activation="sigmoid"))

In [153]:
# Display the Sequential model summary
nn_imported.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_33 (Dense)            (None, 39)                4524      
                                                                 
 dense_34 (Dense)            (None, 13)                520       
                                                                 
 dense_35 (Dense)            (None, 1)                 14        
                                                                 
Total params: 5,058
Trainable params: 5,058
Non-trainable params: 0
_________________________________________________________________


In [154]:
# Compile the Sequential model
nn_imported.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [155]:
# Fit the model using 50 epochs and the training data
fit_model = nn_imported.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 3s 3ms/step - loss: 0.1048 - accuracy: 0.9698
Epoch 2/50
804/804 [==============================] - 2s 3ms/step - loss: 0.0074 - accuracy: 0.9983
Epoch 3/50
804/804 [==============================] - 2s 2ms/step - loss: 0.0042 - accuracy: 0.9986
Epoch 4/50
804/804 [==============================] - 2s 2ms/step - loss: 0.0025 - accuracy: 0.9993
Epoch 5/50
804/804 [==============================] - 2s 2ms/step - loss: 0.0014 - accuracy: 0.9995
Epoch 6/50
804/804 [==============================] - 2s 2ms/step - loss: 0.0018 - accuracy: 0.9993
Epoch 7/50
804/804 [==============================] - 2s 2ms/step - loss: 0.0027 - accuracy: 0.9992
Epoch 8/50
804/804 [==============================] - 2s 2ms/step - loss: 8.6255e-04 - accuracy: 0.9996
Epoch 9/50
804/804 [==============================] - 3s 3ms/step - loss: 4.3941e-04 - accuracy: 0.9998
Epoch 10/50
804/804 [==============================] - 2s 2ms/step - loss: 9.8196e-04 - accu

In [156]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_imported.evaluate (X_test_scaled,y_test,verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.0024 - accuracy: 0.9998 - 441ms/epoch - 2ms/step
Loss: 0.002390462439507246, Accuracy: 0.9997667670249939


In [89]:
# Save model in JSON format
nn_json = nn_imported.to_json()

# Define a relative path to save the model
# The model should be saved with a .json file extension
file_path = "model.json"

# Write the model to the the file
with open(file_path, "w") as json_file:
    json_file.write(nn_json)

# Define a relative path to save the model weights
# The model weights should be saved with a .h5 file extension
file_path = "model.h5"

# Save the weights to the file path
nn_imported.save_weights(file_path)

In [157]:
from tensorflow.keras.models import model_from_json
# Load the model to predict values
# Identify the relative path of the model's location
file_path = "model.json"

# Read in the model and save it as the variable loaded_model
with open(file_path, "r") as json_file:
    model_json = json_file.read()
loaded_model = model_from_json(model_json)

# Identify the relative path for the model's weights
file_path = "model.h5"

# Load the model's weights to the variable loaded_model
loaded_model.load_weights(file_path)

In [158]:
# Define the target set y using the IS_SUCCESSFUL column
y = joined_df["SPECIAL_CONSIDERATIONS_Y"]

# Display a sample of y
y[:5]

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: SPECIAL_CONSIDERATIONS_Y, dtype: float64

In [107]:
# Define features set X by selecting all columns but Attrition_Yes and Attrition_No
X = joined_df.drop(columns=['SPECIAL_CONSIDERATIONS_Y', 'SPECIAL_CONSIDERATIONS_N'])

# Review the features DataFrame
X.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,108590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,6692,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,142590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [159]:
# Split the data into training and testing datasets
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [160]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [161]:
# Define the the number of inputs (features) to the model
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

number_input_features = len(X_train.iloc[0])
# Define the number of hidden nodes for the first hidden layer
# Use the mean of the number of input features plus the number of output nurons
# Use the Python floor division (//) to return the quotent
hidden_nodes_layer1 = (number_input_features + 2)//3

# Define the number of hidden nodes for the second hidden layer
# Use the mean of the number of hidden nodes in the first hidden layer plus the number of output nurons
# Use the Python floor division (//) to return the quotent
hidden_nodes_layer2 = (hidden_nodes_layer1 + 2)//3

# Create the Sequential model instance
loaded_model = Sequential()

# Add the first hidden layer specifying the number of inputs, the number of hidden nodes, and the activation function
loaded_model.add(Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Add the second hidden layer specifying the number of hidden nodes and the activation function
loaded_model.add(Dense(units=hidden_nodes_layer2, activation="relu"))

# Add the output layer to the model specifying the number of output neurons and activation function
loaded_model.add(Dense(units=1, activation="sigmoid"))

In [162]:
# Display the Sequential model summary
loaded_model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_36 (Dense)            (None, 39)                4524      
                                                                 
 dense_37 (Dense)            (None, 13)                520       
                                                                 
 dense_38 (Dense)            (None, 1)                 14        
                                                                 
Total params: 5,058
Trainable params: 5,058
Non-trainable params: 0
_________________________________________________________________


In [163]:
# Compile the Sequential model
loaded_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [164]:
# Fit the model using 20 epochs and the training data
fit_model = loaded_model.fit(X_train_scaled, y_train, epochs=20)

Epoch 1/20
804/804 [==============================] - 2s 2ms/step - loss: 0.0353 - accuracy: 0.9951
Epoch 2/20
804/804 [==============================] - 2s 2ms/step - loss: 0.0062 - accuracy: 0.9991
Epoch 3/20
804/804 [==============================] - 2s 2ms/step - loss: 0.0057 - accuracy: 0.9991
Epoch 4/20
804/804 [==============================] - 2s 3ms/step - loss: 0.0058 - accuracy: 0.9991
Epoch 5/20
804/804 [==============================] - 2s 2ms/step - loss: 0.0058 - accuracy: 0.9991
Epoch 6/20
804/804 [==============================] - 2s 2ms/step - loss: 0.0056 - accuracy: 0.9991
Epoch 7/20
804/804 [==============================] - 2s 2ms/step - loss: 0.0058 - accuracy: 0.9991
Epoch 8/20
804/804 [==============================] - 2s 2ms/step - loss: 0.0057 - accuracy: 0.9991
Epoch 9/20
804/804 [==============================] - 2s 2ms/step - loss: 0.0055 - accuracy: 0.9991
Epoch 10/20
804/804 [==============================] - 2s 3ms/step - loss: 0.0057 - accuracy: 0.9991

In [165]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = loaded_model.evaluate(X_test_scaled,y_test,verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.0056 - accuracy: 0.9993 - 461ms/epoch - 2ms/step
Loss: 0.005589322652667761, Accuracy: 0.9993003010749817


each accuracy is very close to one ,meaning the model fit very well .the first step a change the hidden nodes layers 1&2 and we increase the epochs to 50,the second step we decrease the epoch to 20 and drops the features columns from y and x.

In [166]:
# Set the model's file path
file_path2 = "second.h5"

# Export your model to an HDF5 file
loaded_model.save(file_path)